## Homework

> Note: sometimes your answer doesn't match one of the options exactly. 
> That's fine. 
> Select the option that's closest to your solution.

We recommend using python 3.12 or 3.13 in this homework.

In this homework, we're going to continue working with the lead scoring dataset. You don't need the dataset: we will provide the model for you.


## Question 1

* Install `uv`
* What's the version of uv you installed?
* Use `--version` to find out


## Initialize an empty uv project

You should create an empty folder for homework
and do it there. 

In [1]:
!uv --version

uv 0.8.15
